In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
avocados = "avocado.csv"

avocado_df=pd.read_csv(avocados)

avocado_df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [3]:
census_data = "census_data.csv"

census_df=pd.read_csv(census_data, encoding='latin-1')
census_df.head()

,Id,Country,City,Population,Housing units,Population Density,Housing Unit Density,2017 Median HH Income,2017 Median Age
0,0100000US,United States,United States,308745538(r38234),131704730(r15031),87.4,37.3,57652,37.8
1,0100000US,United States,Alabama,4779736(r38235),2171853(r15032),94.4,42.9,46472,38.7
2,0100000US,United States,Abanda CDP,192,79,64.0,26.4,-,32.7
3,0100000US,United States,Abbeville city,2688,1255,172.9,80.7,40724,44.9
4,0100000US,United States,Adamsville city,4522,1990,180.0,79.2,49077,42.7


In [4]:
#dropping all columns that we do not need in the avocado csv
trim_avocado=avocado_df.drop(['Unnamed: 0','4046','4225','4770','Total Bags','Small Bags','Large Bags','XLarge Bags','year','type'],axis=1)
trim_avocado.head()


,Date,AveragePrice,Total Volume,region
0,2015-12-27,1.33,64236.62,Albany
1,2015-12-20,1.35,54876.98,Albany
2,2015-12-13,0.93,118220.22,Albany
3,2015-12-06,1.08,78992.15,Albany
4,2015-11-29,1.28,51039.60,Albany


In [5]:
#dropping all columns that we do not need in the census csv
trim_census=census_df.drop(['Country','Id','Population','Housing units','Population Density', 'Housing Unit Density'],axis=1)
trim_census.head()

,City,2017 Median HH Income,2017 Median Age
0,United States,57652,37.8
1,Alabama,46472,38.7
2,Abanda CDP,-,32.7
3,Abbeville city,40724,44.9
4,Adamsville city,49077,42.7


In [6]:
trimmed_avocado=trim_avocado.rename(columns={'Date':'date',
                                       'AveragePrice':'average_price',
                                       'Total Volume':'total_volume',
                                       })
trimmed_avocado.head()

,date,average_price,total_volume,region
0,2015-12-27,1.33,64236.62,Albany
1,2015-12-20,1.35,54876.98,Albany
2,2015-12-13,0.93,118220.22,Albany
3,2015-12-06,1.08,78992.15,Albany
4,2015-11-29,1.28,51039.60,Albany


In [7]:
trimmed_census=trim_census.rename(columns={'City':'region',
                                       '2017 Median HH Income':'city_median_income',
                                       '2017 Median Age':'city_median_age',
                                       })
trimmed_census.head()

,region,city_median_income,city_median_age
0,United States,57652,37.8
1,Alabama,46472,38.7
2,Abanda CDP,-,32.7
3,Abbeville city,40724,44.9
4,Adamsville city,49077,42.7


In [8]:
# set index
census_data=trimmed_census.set_index('region')
census_data.head()

,city_median_income,city_median_age
region,,
United States,57652,37.8
Alabama,46472,38.7
Abanda CDP,-,32.7
Abbeville city,40724,44.9
Adamsville city,49077,42.7


In [9]:
census_data.drop(census_data.loc[census_data['city_median_income']=='-'].index, inplace=True)
census_data.drop(census_data.loc[census_data['city_median_income']=='2,500-'].index, inplace=True)
census_data.drop(census_data.loc[census_data['city_median_income']=='250,000+'].index, inplace=True)
census_data.drop(census_data.loc[census_data['city_median_age']=='-'].index, inplace=True)
#census_data[census_data['city_median_income'].apply(lambda x: x not in "r")]
#census_data[census_data['city_median_income'].apply(lambda x: x not in "2,500-")]

census_data.head()

,city_median_income,city_median_age
region,,
United States,57652,37.8
Alabama,46472,38.7
Abbeville city,40724,44.9
Adamsville city,49077,42.7
Addison town,35250,47.6


In [10]:
avocado_data=trimmed_avocado.set_index('region')
avocado_data.head()

,date,average_price,total_volume
region,,,
Albany,2015-12-27,1.33,64236.62
Albany,2015-12-20,1.35,54876.98
Albany,2015-12-13,0.93,118220.22
Albany,2015-12-06,1.08,78992.15
Albany,2015-11-29,1.28,51039.60


In [11]:
avocado_df_volume = avocado_data.groupby(["region"]).sum()["total_volume"]
avocado_df_price = avocado_data.groupby(["region"]).mean()["average_price"]
avocado_df_price.head()

region
Albany                 1.561036
Atlanta                1.337959
BaltimoreWashington    1.534231
Boise                  1.348136
Boston                 1.530888
Name: average_price, dtype: float64

In [12]:
avocado_df = pd.DataFrame({"average_price":avocado_df_price, "total_volume":avocado_df_volume})
avocado_df.head()

,average_price,total_volume
region,,
Albany,1.561036,1.606780e+07
Atlanta,1.337959,8.860512e+07
BaltimoreWashington,1.534231,1.347139e+08
Boise,1.348136,1.441319e+07
Boston,1.530888,9.727398e+07


In [13]:
#avocado_df["average_price"] = avocado_df["average_price"].map("{:,.2f}".format)
#avocado_df["total_volume"] = avocado_df["total_volume"].map("{:,.0f}".format)

#avocado_df.head()

In [14]:
# Connection
connection_string = (f"root:<32j4l0s@127.0.0.1/ETL_db")
engine = create_engine(f'mysql://{connection_string}')

In [15]:
# Confirm tables
engine.table_names()

['avocado', 'census']

In [16]:
avocado_df.to_sql(
    name='avocado', con=engine,
    if_exists='append', index=True)

In [17]:
#census_data['city_median_income'] = pd.to_numeric(census_data['city_median_income'])

census_data.to_sql(
    name='census', con=engine,
    if_exists='append', index=False)